In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
import gc

In [2]:
%%time
training_data_df = pd.read_csv('./Prepped Data/training_data_20220204_2.csv')
print(training_data_df.shape)
training_data_df.head()

(2091509, 18)
Wall time: 3.16 s


,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1992,1992,33.18,1,2.066072,4.435861,2,14,0.034422,0.999407,0,0,0,0,0,0,0,15900
1,서울특별시 강남구 개포동 12 1992,1992,39.53,5,2.066072,4.435861,6,14,0.103102,0.994671,0,0,0,0,0,0,0,24800
2,서울특별시 강남구 개포동 12 1992,1992,39.53,12,2.066072,4.435861,8,14,0.137279,0.990532,0,0,0,0,0,0,0,24700
3,서울특별시 강남구 개포동 12 1992,1992,49.86,11,2.066072,4.435861,8,14,0.137279,0.990532,0,0,0,0,0,0,0,33300
4,서울특별시 강남구 개포동 12 1992,1992,39.53,4,2.066072,4.435861,9,14,0.154309,0.988023,0,0,0,0,0,0,0,24700


In [3]:
basedir = './Training/Training5_20220204/cat_col_maps/'

In [4]:
filenames = [f for f in os.listdir(basedir)]
filenames

['구.xlsx',
 '대장구분명.xlsx',
 '동.xlsx',
 '부동산유형.xlsx',
 '브랜드.xlsx',
 '일괄계약.xlsx',
 '지목.xlsx']

In [5]:
cat_cols = [col.replace('.xlsx', '') for col in filenames]
cat_cols

['구', '대장구분명', '동', '부동산유형', '브랜드', '일괄계약', '지목']

# Training - Catboost Regressor

In [6]:
import catboost

In [7]:
from sklearn.model_selection import GroupKFold

In [8]:
gkfold = GroupKFold(n_splits=5)

In [9]:
#count = 0
models_list = []
for i, (train_idx, val_idx) in enumerate(gkfold.split(training_data_df, groups=training_data_df['주소_건축년도'])):
    print('-------------- Fold {} starting ---------------'.format(i))
    
    print(i)
    print(len(train_idx), len(val_idx))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]

    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]

    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)

    model = catboost.CatBoostRegressor(
        iterations=100000, depth=6, task_type='GPU', loss_function='RMSE'
    )

    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)

    model.save_model('./Training/Training5_20220204/catboost_RMSE_{}'.format(i))
    
    print('-------------- Fold {} ending ---------------'.format(i))
    #count += 1

-------------- Fold 0 starting ---------------
0
1673207 418302
Learning rate set to 0.037261
0:	learn: 36855.6118116	test: 38503.8746422	best: 38503.8746422 (0)	total: 60.4ms	remaining: 1h 40m 43s
50:	learn: 17807.6393808	test: 20737.6752127	best: 20737.6752127 (50)	total: 2.79s	remaining: 1h 31m 4s
100:	learn: 14447.9003920	test: 17723.3182868	best: 17723.3182868 (100)	total: 5.53s	remaining: 1h 31m 13s
150:	learn: 13176.9156446	test: 16657.6758203	best: 16657.6758203 (150)	total: 8.25s	remaining: 1h 30m 56s
200:	learn: 12420.9021650	test: 16082.9331569	best: 16082.9331569 (200)	total: 11s	remaining: 1h 31m 7s
250:	learn: 11886.9063192	test: 15686.5196116	best: 15686.5196116 (250)	total: 13.9s	remaining: 1h 31m 54s
300:	learn: 11477.1154076	test: 15443.2880932	best: 15443.2880932 (300)	total: 16.6s	remaining: 1h 31m 31s
350:	learn: 11158.6742735	test: 15279.2660215	best: 15279.2660215 (350)	total: 19.3s	remaining: 1h 31m 21s
400:	learn: 10901.8478056	test: 15128.7480218	best: 15128.7

1100:	learn: 9425.8888398	test: 11481.2610641	best: 11480.3667353 (1099)	total: 1m	remaining: 1h 30m 17s
1150:	learn: 9363.9146140	test: 11469.1323451	best: 11469.1323451 (1150)	total: 1m 3s	remaining: 1h 30m 12s
1200:	learn: 9297.4393665	test: 11427.6753109	best: 11427.6753109 (1200)	total: 1m 5s	remaining: 1h 30m 3s
1250:	learn: 9235.4619774	test: 11406.2736376	best: 11405.9725502 (1248)	total: 1m 8s	remaining: 1h 29m 53s
1300:	learn: 9177.9370036	test: 11392.0809244	best: 11392.0809244 (1300)	total: 1m 11s	remaining: 1h 29m 48s
1350:	learn: 9126.5550337	test: 11381.4606755	best: 11381.4606755 (1350)	total: 1m 13s	remaining: 1h 29m 44s
1400:	learn: 9071.9729864	test: 11356.2226669	best: 11356.2226669 (1400)	total: 1m 16s	remaining: 1h 29m 45s
1450:	learn: 9014.1118741	test: 11338.4734230	best: 11338.4734230 (1450)	total: 1m 19s	remaining: 1h 29m 40s
1500:	learn: 8965.5696597	test: 11326.9358626	best: 11326.7446507 (1499)	total: 1m 21s	remaining: 1h 29m 40s
1550:	learn: 8922.1370044	t

4900:	learn: 7375.1183070	test: 10931.1222776	best: 10930.7434315 (4898)	total: 4m 29s	remaining: 1h 27m 17s
4950:	learn: 7363.7479644	test: 10930.2659566	best: 10929.6357130 (4944)	total: 4m 32s	remaining: 1h 27m 14s
5000:	learn: 7349.9085803	test: 10922.1934289	best: 10922.1934289 (5000)	total: 4m 35s	remaining: 1h 27m 10s
5050:	learn: 7335.4727645	test: 10916.8041412	best: 10916.8041412 (5050)	total: 4m 38s	remaining: 1h 27m 6s
5100:	learn: 7322.6706682	test: 10913.6849627	best: 10913.1024581 (5081)	total: 4m 40s	remaining: 1h 27m 3s
5150:	learn: 7310.0210419	test: 10912.8061406	best: 10912.2396693 (5110)	total: 4m 43s	remaining: 1h 27m 2s
5200:	learn: 7298.1096460	test: 10908.0837287	best: 10908.0837287 (5200)	total: 4m 46s	remaining: 1h 27m 1s
5250:	learn: 7286.6194809	test: 10910.2395563	best: 10907.1515954 (5203)	total: 4m 49s	remaining: 1h 26m 58s
5300:	learn: 7273.3026690	test: 10912.0186778	best: 10907.1515954 (5203)	total: 4m 51s	remaining: 1h 26m 54s
5350:	learn: 7263.05175

2750:	learn: 8012.7650255	test: 13247.2216110	best: 13243.5902083 (2747)	total: 2m 28s	remaining: 1h 27m 36s
2800:	learn: 7990.2838028	test: 13236.1562403	best: 13236.1562403 (2800)	total: 2m 31s	remaining: 1h 27m 35s
2850:	learn: 7963.8983852	test: 13226.4333749	best: 13226.3105619 (2844)	total: 2m 34s	remaining: 1h 27m 34s
2900:	learn: 7941.1803390	test: 13232.5468068	best: 13225.2688801 (2857)	total: 2m 36s	remaining: 1h 27m 32s
2950:	learn: 7916.1294988	test: 13226.9610029	best: 13225.2688801 (2857)	total: 2m 39s	remaining: 1h 27m 28s
3000:	learn: 7893.1729766	test: 13219.9060287	best: 13218.9761049 (2992)	total: 2m 42s	remaining: 1h 27m 26s
3050:	learn: 7867.7523167	test: 13213.4187124	best: 13211.5556093 (3042)	total: 2m 44s	remaining: 1h 27m 22s
3100:	learn: 7847.2446946	test: 13205.7627539	best: 13204.9700336 (3099)	total: 2m 47s	remaining: 1h 27m 19s
3150:	learn: 7826.6586765	test: 13195.1504788	best: 13195.1504788 (3150)	total: 2m 50s	remaining: 1h 27m 17s
3200:	learn: 7805.3

2200:	learn: 8432.8880219	test: 11532.8891520	best: 11532.4483454 (2160)	total: 1m 58s	remaining: 1h 28m 6s
2250:	learn: 8400.5189818	test: 11528.7747363	best: 11525.5558336 (2216)	total: 2m 1s	remaining: 1h 28m 4s
2300:	learn: 8368.6792515	test: 11515.1466357	best: 11514.5531947 (2292)	total: 2m 4s	remaining: 1h 28m 2s
2350:	learn: 8338.2389576	test: 11507.7466820	best: 11507.5824362 (2349)	total: 2m 7s	remaining: 1h 28m
2400:	learn: 8310.2234526	test: 11501.6213868	best: 11500.2029002 (2394)	total: 2m 9s	remaining: 1h 27m 58s
2450:	learn: 8277.5082939	test: 11497.3366224	best: 11495.2158842 (2423)	total: 2m 12s	remaining: 1h 27m 56s
2500:	learn: 8251.4167304	test: 11482.9870790	best: 11480.9248256 (2494)	total: 2m 15s	remaining: 1h 27m 57s
2550:	learn: 8219.9975951	test: 11489.9793041	best: 11480.6431637 (2505)	total: 2m 18s	remaining: 1h 27m 53s
2600:	learn: 8191.0100328	test: 11492.6864480	best: 11480.6431637 (2505)	total: 2m 20s	remaining: 1h 27m 50s
2650:	learn: 8163.6625864	test

6000:	learn: 7071.7856928	test: 11214.8481279	best: 11214.7582724 (5992)	total: 5m 26s	remaining: 1h 25m 13s
6050:	learn: 7061.4310934	test: 11209.4801257	best: 11209.4801257 (6050)	total: 5m 29s	remaining: 1h 25m 10s
6100:	learn: 7048.6092427	test: 11219.2912743	best: 11208.4724165 (6059)	total: 5m 31s	remaining: 1h 25m 7s
6150:	learn: 7039.8451690	test: 11215.3765169	best: 11208.4724165 (6059)	total: 5m 34s	remaining: 1h 25m 5s
6200:	learn: 7028.7595157	test: 11205.6406847	best: 11205.0746999 (6199)	total: 5m 37s	remaining: 1h 25m 3s
6250:	learn: 7020.1249209	test: 11205.3355486	best: 11205.0554604 (6208)	total: 5m 40s	remaining: 1h 25m
6300:	learn: 7008.1393010	test: 11204.0312443	best: 11200.9087986 (6259)	total: 5m 42s	remaining: 1h 24m 58s
6350:	learn: 6998.7002861	test: 11204.9874507	best: 11200.9087986 (6259)	total: 5m 45s	remaining: 1h 24m 55s
6400:	learn: 6989.0569981	test: 11205.4371125	best: 11200.9087986 (6259)	total: 5m 48s	remaining: 1h 24m 52s
6450:	learn: 6980.2932094	

In [10]:
#count = 0
models_list = []
for i, (train_idx, val_idx) in enumerate(gkfold.split(training_data_df, groups=training_data_df['주소_건축년도'])):
    print('-------------- Fold {} starting ---------------'.format(i))
    
    print(i)
    print(len(train_idx), len(val_idx))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]

    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]

    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)

    model = catboost.CatBoostRegressor(
        iterations=100000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.5'
    )

    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)

    model.save_model('./Training/Training5_20220204/catboost_quantile05_{}'.format(i))
    
    print('-------------- Fold {} ending ---------------'.format(i))
    #count += 1

-------------- Fold 0 starting ---------------
0
1673207 418302
0:	learn: 10904.1418832	test: 11132.0605687	best: 11132.0605687 (0)	total: 116ms	remaining: 3h 13m 11s
50:	learn: 5732.0118599	test: 5931.9922544	best: 5931.9922544 (50)	total: 3.27s	remaining: 1h 46m 55s
100:	learn: 4534.4827747	test: 4720.6654714	best: 4720.6654714 (100)	total: 6.83s	remaining: 1h 52m 36s
150:	learn: 4043.5300115	test: 4289.3748536	best: 4289.3748536 (150)	total: 10.5s	remaining: 1h 55m 35s
200:	learn: 3778.4028205	test: 4072.6455432	best: 4072.6455432 (200)	total: 14s	remaining: 1h 55m 36s
250:	learn: 3624.8285550	test: 3941.9255179	best: 3941.9255179 (250)	total: 17.4s	remaining: 1h 55m 8s
300:	learn: 3497.3951890	test: 3837.6248356	best: 3837.6248356 (300)	total: 20.7s	remaining: 1h 54m 11s
350:	learn: 3388.6801000	test: 3751.8331158	best: 3751.8331158 (350)	total: 24s	remaining: 1h 53m 45s
400:	learn: 3312.1399086	test: 3694.9307629	best: 3694.9307629 (400)	total: 27.3s	remaining: 1h 53m 7s
450:	lear

3850:	learn: 2227.3929478	test: 2998.3344091	best: 2998.3344091 (3850)	total: 4m 18s	remaining: 1h 47m 29s
3900:	learn: 2222.7295200	test: 2995.9598568	best: 2995.9598568 (3900)	total: 4m 21s	remaining: 1h 47m 27s
3950:	learn: 2217.9306876	test: 2992.8873206	best: 2992.8873206 (3950)	total: 4m 25s	remaining: 1h 47m 24s
4000:	learn: 2213.2998487	test: 2990.6568938	best: 2990.6568938 (4000)	total: 4m 28s	remaining: 1h 47m 21s
4050:	learn: 2208.9520591	test: 2987.9644085	best: 2987.9644085 (4050)	total: 4m 31s	remaining: 1h 47m 18s
4100:	learn: 2204.0606620	test: 2987.1553471	best: 2986.6525907 (4081)	total: 4m 35s	remaining: 1h 47m 16s
4150:	learn: 2199.5499278	test: 2986.0571166	best: 2986.0537506 (4149)	total: 4m 38s	remaining: 1h 47m 14s
4200:	learn: 2195.6917369	test: 2984.6094353	best: 2984.5167176 (4196)	total: 4m 41s	remaining: 1h 47m 10s
4250:	learn: 2191.1142949	test: 2983.3545333	best: 2983.3545333 (4250)	total: 4m 45s	remaining: 1h 47m 7s
4300:	learn: 2185.9747324	test: 2981.1

2150:	learn: 2463.3347004	test: 3014.1897481	best: 3014.1897481 (2150)	total: 2m 23s	remaining: 1h 48m 36s
2200:	learn: 2454.1787454	test: 3010.0101075	best: 3010.0101075 (2200)	total: 2m 26s	remaining: 1h 48m 33s
2250:	learn: 2445.1580420	test: 3009.6878906	best: 3008.5713002 (2234)	total: 2m 29s	remaining: 1h 48m 30s
2300:	learn: 2436.6751322	test: 3003.7212158	best: 3003.7212158 (2300)	total: 2m 33s	remaining: 1h 48m 24s
2350:	learn: 2427.8181385	test: 2997.8968305	best: 2997.7227171 (2348)	total: 2m 36s	remaining: 1h 48m 19s
2400:	learn: 2419.4593114	test: 2994.0369589	best: 2993.9742291 (2398)	total: 2m 39s	remaining: 1h 48m 14s
2450:	learn: 2410.5617727	test: 2989.6541351	best: 2989.6541351 (2450)	total: 2m 43s	remaining: 1h 48m 9s
2500:	learn: 2401.5865102	test: 2986.1115845	best: 2986.1115845 (2500)	total: 2m 46s	remaining: 1h 48m 6s
2550:	learn: 2393.4825805	test: 2982.4371483	best: 2982.4371483 (2550)	total: 2m 49s	remaining: 1h 48m
2600:	learn: 2385.3322918	test: 2977.489756

2050:	learn: 2467.6834080	test: 3311.9757496	best: 3311.6440466 (2047)	total: 2m 16s	remaining: 1h 48m 38s
2100:	learn: 2456.8626643	test: 3304.0641450	best: 3304.0641450 (2100)	total: 2m 19s	remaining: 1h 48m 38s
2150:	learn: 2446.0856786	test: 3298.2813757	best: 3297.8389011 (2149)	total: 2m 23s	remaining: 1h 48m 32s
2200:	learn: 2436.0193760	test: 3294.6705873	best: 3294.6705873 (2200)	total: 2m 26s	remaining: 1h 48m 30s
2250:	learn: 2426.9798537	test: 3289.9499405	best: 3289.9499405 (2250)	total: 2m 29s	remaining: 1h 48m 27s
2300:	learn: 2417.9362004	test: 3285.5126488	best: 3285.4753169 (2299)	total: 2m 33s	remaining: 1h 48m 22s
2350:	learn: 2409.7911137	test: 3280.8443278	best: 3280.8443278 (2350)	total: 2m 36s	remaining: 1h 48m 21s
2400:	learn: 2402.2200816	test: 3277.3247271	best: 3277.2289494 (2399)	total: 2m 39s	remaining: 1h 48m 17s
2450:	learn: 2392.5961008	test: 3269.8644711	best: 3269.8644711 (2450)	total: 2m 43s	remaining: 1h 48m 12s
2500:	learn: 2384.3765033	test: 3266.

5900:	learn: 2051.6752034	test: 3118.9990007	best: 3118.2269652 (5875)	total: 6m 33s	remaining: 1h 44m 28s
5950:	learn: 2048.6022064	test: 3118.0675397	best: 3118.0675397 (5950)	total: 6m 36s	remaining: 1h 44m 23s
6000:	learn: 2045.2328493	test: 3116.8973995	best: 3116.8973995 (6000)	total: 6m 39s	remaining: 1h 44m 21s
6050:	learn: 2042.1722453	test: 3116.0023524	best: 3116.0023524 (6050)	total: 6m 43s	remaining: 1h 44m 18s
6100:	learn: 2038.9031028	test: 3114.9720537	best: 3114.8245622 (6096)	total: 6m 46s	remaining: 1h 44m 13s
6150:	learn: 2035.6741993	test: 3113.8762712	best: 3113.8762712 (6150)	total: 6m 49s	remaining: 1h 44m 11s
6200:	learn: 2032.7555790	test: 3112.7731448	best: 3112.2293845 (6194)	total: 6m 53s	remaining: 1h 44m 7s
6250:	learn: 2030.0476391	test: 3112.3037423	best: 3112.2293845 (6194)	total: 6m 56s	remaining: 1h 44m 3s
6300:	learn: 2026.9270592	test: 3110.7312133	best: 3110.2058130 (6280)	total: 6m 59s	remaining: 1h 44m
6350:	learn: 2024.3568190	test: 3110.267624

...

4400:	learn: 2181.1781949	test: 2818.7928530	best: 2818.7900990 (4399)	total: 4m 51s	remaining: 1h 45m 35s
4450:	learn: 2177.1497155	test: 2816.9528809	best: 2816.9528809 (4450)	total: 4m 54s	remaining: 1h 45m 32s
4500:	learn: 2172.9396255	test: 2815.4638897	best: 2815.4638897 (4500)	total: 4m 58s	remaining: 1h 45m 28s
4550:	learn: 2169.1922063	test: 2815.1744147	best: 2815.1744147 (4550)	total: 5m 1s	remaining: 1h 45m 25s
4600:	learn: 2165.5534169	test: 2813.4853001	best: 2813.4853001 (4600)	total: 5m 4s	remaining: 1h 45m 21s
4650:	learn: 2161.2668271	test: 2811.8387194	best: 2811.8176055 (4649)	total: 5m 8s	remaining: 1h 45m 18s
4700:	learn: 2156.7762889	test: 2810.5801454	best: 2810.5599495 (4699)	total: 5m 11s	remaining: 1h 45m 14s
4750:	learn: 2152.3796924	test: 2810.2267166	best: 2810.1193109 (4717)	total: 5m 14s	remaining: 1h 45m 11s
4800:	learn: 2148.4970216	test: 2808.5455580	best: 2808.5455580 (4800)	total: 5m 18s	remaining: 1h 45m 8s
4850:	learn: 2144.5256110	test: 2806.5495

8250:	learn: 1953.0271819	test: 2764.9257044	best: 2764.9257044 (8250)	total: 9m 7s	remaining: 1h 41m 28s
8300:	learn: 1950.9754573	test: 2764.4801698	best: 2764.4152980 (8296)	total: 9m 10s	remaining: 1h 41m 25s
8350:	learn: 1948.9632066	test: 2764.0162753	best: 2763.9728235 (8349)	total: 9m 14s	remaining: 1h 41m 22s
8400:	learn: 1946.7200795	test: 2763.5508508	best: 2763.4896510 (8396)	total: 9m 17s	remaining: 1h 41m 19s
8450:	learn: 1944.7805035	test: 2763.1506041	best: 2763.0447284 (8439)	total: 9m 20s	remaining: 1h 41m 16s
8500:	learn: 1942.8250157	test: 2762.3424607	best: 2762.2668790 (8490)	total: 9m 24s	remaining: 1h 41m 13s
8550:	learn: 1941.1264141	test: 2761.9235481	best: 2761.8684682 (8548)	total: 9m 27s	remaining: 1h 41m 10s
8600:	learn: 1939.4171026	test: 2761.5474753	best: 2761.5474753 (8600)	total: 9m 30s	remaining: 1h 41m 7s
8650:	learn: 1937.6158383	test: 2761.0159550	best: 2760.9777051 (8646)	total: 9m 34s	remaining: 1h 41m 4s
8700:	learn: 1935.2992714	test: 2760.038

12050:	learn: 1827.9874947	test: 2729.6057681	best: 2729.1161697 (11915)	total: 13m 21s	remaining: 1h 37m 31s
12100:	learn: 1826.5861713	test: 2729.0687398	best: 2729.0528279 (12099)	total: 13m 25s	remaining: 1h 37m 27s
12150:	learn: 1825.3205587	test: 2730.1623803	best: 2728.7064370 (12149)	total: 13m 28s	remaining: 1h 37m 24s
12200:	learn: 1824.2076396	test: 2729.8735172	best: 2728.7064370 (12149)	total: 13m 31s	remaining: 1h 37m 21s
12250:	learn: 1822.9753808	test: 2729.5822062	best: 2728.7064370 (12149)	total: 13m 34s	remaining: 1h 37m 17s
12300:	learn: 1821.8225288	test: 2728.9536842	best: 2728.7064370 (12149)	total: 13m 38s	remaining: 1h 37m 14s
12350:	learn: 1820.4582314	test: 2728.9965240	best: 2728.6339152 (12309)	total: 13m 41s	remaining: 1h 37m 10s
12400:	learn: 1819.3168544	test: 2728.6121893	best: 2728.6039273 (12396)	total: 13m 45s	remaining: 1h 37m 8s
12450:	learn: 1818.2027113	test: 2728.0448671	best: 2728.0448671 (12450)	total: 13m 48s	remaining: 1h 37m 4s
12500:	learn

1600:	learn: 2607.6807665	test: 3101.7866273	best: 3101.7734693 (1599)	total: 1m 46s	remaining: 1h 48m 39s
1650:	learn: 2594.5732987	test: 3095.4322366	best: 3095.2948427 (1649)	total: 1m 49s	remaining: 1h 48m 33s
1700:	learn: 2582.3985159	test: 3090.5952986	best: 3090.5175747 (1698)	total: 1m 52s	remaining: 1h 48m 33s
1750:	learn: 2569.6163250	test: 3084.6408495	best: 3084.6408495 (1750)	total: 1m 56s	remaining: 1h 48m 28s
1800:	learn: 2559.0688450	test: 3079.7240456	best: 3079.7240456 (1800)	total: 1m 59s	remaining: 1h 48m 28s
1850:	learn: 2545.6492582	test: 3074.6756618	best: 3074.6756618 (1850)	total: 2m 2s	remaining: 1h 48m 26s
1900:	learn: 2532.4946665	test: 3070.8589269	best: 3070.7894650 (1899)	total: 2m 6s	remaining: 1h 48m 24s
1950:	learn: 2519.4586495	test: 3066.7551643	best: 3066.7496563 (1949)	total: 2m 9s	remaining: 1h 48m 21s
2000:	learn: 2507.9015257	test: 3062.6841437	best: 3062.4292459 (1999)	total: 2m 12s	remaining: 1h 48m 16s
2050:	learn: 2498.2966756	test: 3059.178

5450:	learn: 2101.5250417	test: 2954.4835513	best: 2954.4835513 (5450)	total: 6m 1s	remaining: 1h 44m 31s
5500:	learn: 2098.5801072	test: 2952.8231967	best: 2952.8231967 (5500)	total: 6m 4s	remaining: 1h 44m 29s
5550:	learn: 2094.9908917	test: 2952.4504890	best: 2952.4373310 (5549)	total: 6m 8s	remaining: 1h 44m 24s
5600:	learn: 2091.5742597	test: 2951.7445476	best: 2951.7445476 (5600)	total: 6m 11s	remaining: 1h 44m 21s
5650:	learn: 2088.6186153	test: 2950.3097148	best: 2950.3097148 (5650)	total: 6m 14s	remaining: 1h 44m 18s
5700:	learn: 2085.4301055	test: 2948.9948339	best: 2948.9948339 (5700)	total: 6m 18s	remaining: 1h 44m 15s
5750:	learn: 2082.1112402	test: 2947.7943969	best: 2947.5459251 (5744)	total: 6m 21s	remaining: 1h 44m 13s
5800:	learn: 2078.7011872	test: 2946.4984879	best: 2946.4813519 (5799)	total: 6m 25s	remaining: 1h 44m 11s
5850:	learn: 2075.8809473	test: 2945.7181886	best: 2945.6083347 (5843)	total: 6m 28s	remaining: 1h 44m 8s
5900:	learn: 2073.0241405	test: 2945.3907

In [11]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.25')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training5_20220204/catboost_quantile025_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 6802.1755109	test: 6935.6861598	best: 6935.6861598 (0)	total: 281ms	remaining: 46m 50s
50:	learn: 4257.0513559	test: 4379.7847106	best: 4379.7847106 (50)	total: 3.38s	remaining: 11m
100:	learn: 3452.6167964	test: 3563.3285425	best: 3563.3285425 (100)	total: 6.75s	remaining: 11m 1s
150:	learn: 3083.0376971	test: 3212.5294357	best: 3212.5294357 (150)	total: 10.3s	remaining: 11m 9s
200:	learn: 2854.9480369	test: 3007.9277842	best: 3007.9277842 (200)	total: 13.8s	remaining: 11m 14s
250:	learn: 2742.7880089	test: 2915.0684051	best: 2915.0684051 (250)	total: 17.4s	remaining: 11m 14s
300:	learn: 2652.3012921	test: 2843.2262624	best: 2843.2262624 (300)	total: 20.8s	remaining: 11m 11s
350:	learn: 2590.0947151	test: 2791.2608976	best: 2791.2608976 (350)	total: 24.3s	remaining: 11m 6s
400:	learn: 2534.9383597	test: 2745.2175318	best: 2745.2175318 (400)	total: 27.6s	remaining: 11m 1s
450:	learn: 2482.4558253	test: 2706.3917648	best: 2706.391

4000:	learn: 1725.2074131	test: 2243.9100554	best: 2243.9100554 (4000)	total: 4m 27s	remaining: 6m 40s
4050:	learn: 1721.1562897	test: 2242.6066526	best: 2242.5459118 (4041)	total: 4m 30s	remaining: 6m 37s
4100:	learn: 1717.3700086	test: 2240.6488709	best: 2240.6488709 (4100)	total: 4m 33s	remaining: 6m 34s
4150:	learn: 1713.7810970	test: 2239.2219975	best: 2239.2219975 (4150)	total: 4m 37s	remaining: 6m 30s
4200:	learn: 1709.7870425	test: 2237.0991293	best: 2237.0991293 (4200)	total: 4m 40s	remaining: 6m 27s
4250:	learn: 1705.7392851	test: 2236.9828497	best: 2236.7053086 (4212)	total: 4m 44s	remaining: 6m 24s
4300:	learn: 1701.7417116	test: 2234.3290733	best: 2234.3290733 (4300)	total: 4m 47s	remaining: 6m 20s
4350:	learn: 1698.5069941	test: 2233.7854660	best: 2233.4138302 (4336)	total: 4m 50s	remaining: 6m 17s
4400:	learn: 1695.2376986	test: 2232.1473003	best: 2232.1473003 (4400)	total: 4m 54s	remaining: 6m 14s
4450:	learn: 1692.3313182	test: 2230.6301572	best: 2230.6301572 (4450)	to

1200:	learn: 2142.4930113	test: 2441.5185584	best: 2441.5185584 (1200)	total: 1m 19s	remaining: 9m 45s
1250:	learn: 2125.9762456	test: 2430.1145871	best: 2430.1145871 (1250)	total: 1m 23s	remaining: 9m 41s
1300:	learn: 2111.8276077	test: 2420.0761364	best: 2420.0761364 (1300)	total: 1m 26s	remaining: 9m 38s
1350:	learn: 2097.9461071	test: 2410.4047315	best: 2410.4047315 (1350)	total: 1m 29s	remaining: 9m 35s
1400:	learn: 2087.3866868	test: 2402.8454848	best: 2402.8454848 (1400)	total: 1m 33s	remaining: 9m 31s
1450:	learn: 2074.3306190	test: 2393.4582001	best: 2393.4582001 (1450)	total: 1m 36s	remaining: 9m 27s
1500:	learn: 2063.9353672	test: 2387.1244794	best: 2387.1244794 (1500)	total: 1m 39s	remaining: 9m 24s
1550:	learn: 2053.4348518	test: 2379.1346922	best: 2379.1346922 (1550)	total: 1m 43s	remaining: 9m 21s
1600:	learn: 2039.4118265	test: 2369.9212148	best: 2369.8306391 (1599)	total: 1m 46s	remaining: 9m 18s
1650:	learn: 2029.0815996	test: 2363.0142050	best: 2363.0142050 (1650)	to

5200:	learn: 1655.5817326	test: 2186.4494074	best: 2186.4494074 (5200)	total: 5m 44s	remaining: 5m 17s
5250:	learn: 1652.6560742	test: 2185.4285947	best: 2185.4285947 (5250)	total: 5m 47s	remaining: 5m 14s
5300:	learn: 1650.5791047	test: 2184.5356895	best: 2184.5304875 (5299)	total: 5m 50s	remaining: 5m 11s
5350:	learn: 1647.8928907	test: 2184.7425449	best: 2183.8051169 (5344)	total: 5m 54s	remaining: 5m 7s
5400:	learn: 1645.9866687	test: 2184.1821077	best: 2183.8051169 (5344)	total: 5m 57s	remaining: 5m 4s
5450:	learn: 1642.8709849	test: 2182.7962381	best: 2182.7962381 (5450)	total: 6m 1s	remaining: 5m 1s
5500:	learn: 1640.1449910	test: 2181.9677458	best: 2181.5419482 (5495)	total: 6m 4s	remaining: 4m 58s
5550:	learn: 1637.8721820	test: 2181.6620528	best: 2181.5419482 (5495)	total: 6m 7s	remaining: 4m 54s
5600:	learn: 1635.3337656	test: 2181.0391918	best: 2180.9932919 (5572)	total: 6m 11s	remaining: 4m 51s
5650:	learn: 1632.8866901	test: 2180.2478783	best: 2180.2478783 (5650)	total: 6

9200:	learn: 1503.6067719	test: 2138.3480452	best: 2138.1393539 (9193)	total: 10m 12s	remaining: 53.2s
9250:	learn: 1501.9155143	test: 2137.6693394	best: 2137.6693394 (9250)	total: 10m 15s	remaining: 49.9s
9300:	learn: 1500.3997473	test: 2137.1339941	best: 2137.1339941 (9300)	total: 10m 19s	remaining: 46.5s
9350:	learn: 1498.9240662	test: 2136.7133985	best: 2136.6407237 (9344)	total: 10m 22s	remaining: 43.2s
9400:	learn: 1497.6010786	test: 2136.0322446	best: 2136.0322446 (9400)	total: 10m 25s	remaining: 39.9s
9450:	learn: 1496.3360780	test: 2135.6971757	best: 2135.6971757 (9450)	total: 10m 29s	remaining: 36.5s
9500:	learn: 1495.2011269	test: 2134.9453361	best: 2134.9335552 (9497)	total: 10m 32s	remaining: 33.2s
9550:	learn: 1494.0057411	test: 2134.5502914	best: 2134.5502914 (9550)	total: 10m 35s	remaining: 29.9s
9600:	learn: 1492.5967678	test: 2134.0561030	best: 2134.0515130 (9599)	total: 10m 39s	remaining: 26.6s
9650:	learn: 1491.1590305	test: 2133.2813135	best: 2133.2813135 (9650)	to

...

1000:	learn: 2176.8167884	test: 2382.7245196	best: 2382.5387782 (999)	total: 1m 6s	remaining: 9m 59s
1050:	learn: 2162.1543778	test: 2374.2102500	best: 2374.2102500 (1050)	total: 1m 9s	remaining: 9m 55s
1100:	learn: 2148.0530168	test: 2366.9723023	best: 2366.9723023 (1100)	total: 1m 13s	remaining: 9m 51s
1150:	learn: 2134.3587876	test: 2359.8677702	best: 2359.8677702 (1150)	total: 1m 16s	remaining: 9m 47s
1200:	learn: 2121.7776306	test: 2352.4427041	best: 2352.4427041 (1200)	total: 1m 19s	remaining: 9m 43s
1250:	learn: 2108.5856992	test: 2343.7351961	best: 2343.7351961 (1250)	total: 1m 22s	remaining: 9m 40s
1300:	learn: 2095.8562856	test: 2337.2960588	best: 2337.2960588 (1300)	total: 1m 26s	remaining: 9m 36s
1350:	learn: 2084.2618230	test: 2330.7611439	best: 2330.7611439 (1350)	total: 1m 29s	remaining: 9m 32s
1400:	learn: 2071.2784300	test: 2324.4627661	best: 2324.4627661 (1400)	total: 1m 32s	remaining: 9m 29s
1450:	learn: 2058.5193344	test: 2317.9784940	best: 2317.9784940 (1450)	total

5000:	learn: 1671.9421327	test: 2144.5452902	best: 2144.5380993 (4998)	total: 5m 32s	remaining: 5m 32s
5050:	learn: 1669.5221380	test: 2143.8234386	best: 2143.8234386 (5050)	total: 5m 35s	remaining: 5m 29s
5100:	learn: 1666.8600980	test: 2143.3196112	best: 2143.3196112 (5100)	total: 5m 39s	remaining: 5m 25s
5150:	learn: 1663.4879868	test: 2142.2417297	best: 2142.2417297 (5150)	total: 5m 42s	remaining: 5m 22s
5200:	learn: 1660.9908804	test: 2141.7042424	best: 2141.6150437 (5194)	total: 5m 45s	remaining: 5m 19s
5250:	learn: 1658.3724453	test: 2140.8922740	best: 2140.7929773 (5244)	total: 5m 49s	remaining: 5m 15s
5300:	learn: 1655.4931458	test: 2139.3061281	best: 2139.3061281 (5300)	total: 5m 52s	remaining: 5m 12s
5350:	learn: 1652.6377143	test: 2137.9326515	best: 2137.9326515 (5350)	total: 5m 55s	remaining: 5m 9s
5400:	learn: 1649.5443684	test: 2136.4202514	best: 2136.3977605 (5396)	total: 5m 59s	remaining: 5m 5s
5450:	learn: 1646.9543912	test: 2136.3230967	best: 2136.1431693 (5427)	tota

1500:	learn: 2058.3404717	test: 2350.9337821	best: 2350.9337821 (1500)	total: 1m 39s	remaining: 9m 24s
1550:	learn: 2044.5611831	test: 2344.7287193	best: 2344.7287193 (1550)	total: 1m 42s	remaining: 9m 21s
1600:	learn: 2030.6824447	test: 2337.8769259	best: 2337.8530580 (1598)	total: 1m 46s	remaining: 9m 17s
1650:	learn: 2017.3216097	test: 2331.6937421	best: 2331.6937421 (1650)	total: 1m 49s	remaining: 9m 14s
1700:	learn: 2008.0928325	test: 2328.1453690	best: 2328.1453690 (1700)	total: 1m 52s	remaining: 9m 10s
1750:	learn: 1997.8885303	test: 2324.0709824	best: 2324.0709824 (1750)	total: 1m 56s	remaining: 9m 7s
1800:	learn: 1989.7331975	test: 2319.7629649	best: 2319.7629649 (1800)	total: 1m 59s	remaining: 9m 3s
1850:	learn: 1979.6151871	test: 2315.3509076	best: 2315.3509076 (1850)	total: 2m 2s	remaining: 8m 59s
1900:	learn: 1969.6925571	test: 2311.0741021	best: 2311.0741021 (1900)	total: 2m 5s	remaining: 8m 56s
1950:	learn: 1961.4810735	test: 2307.4648351	best: 2307.4648351 (1950)	total:

5500:	learn: 1643.0679844	test: 2201.1612882	best: 2201.1580752 (5499)	total: 6m 5s	remaining: 4m 59s
5550:	learn: 1640.4839757	test: 2200.1002340	best: 2200.0199091 (5544)	total: 6m 9s	remaining: 4m 55s
5600:	learn: 1638.0863204	test: 2199.5368885	best: 2199.5368885 (5600)	total: 6m 12s	remaining: 4m 52s
5650:	learn: 1635.7714379	test: 2198.2056366	best: 2198.1992106 (5649)	total: 6m 15s	remaining: 4m 49s
5700:	learn: 1633.2443450	test: 2197.2636164	best: 2197.2204704 (5694)	total: 6m 19s	remaining: 4m 45s
5750:	learn: 1631.0489551	test: 2196.7414278	best: 2196.5745050 (5739)	total: 6m 22s	remaining: 4m 42s
5800:	learn: 1628.6617037	test: 2195.9992254	best: 2195.9834665 (5799)	total: 6m 25s	remaining: 4m 39s
5850:	learn: 1626.2814904	test: 2196.2005733	best: 2195.8295486 (5818)	total: 6m 28s	remaining: 4m 35s
5900:	learn: 1623.9562039	test: 2195.5714378	best: 2195.5663888 (5899)	total: 6m 32s	remaining: 4m 32s
5950:	learn: 1621.6508073	test: 2195.6029558	best: 2195.4323609 (5917)	tota

9500:	learn: 1497.0599423	test: 2159.5919876	best: 2159.5500656 (9498)	total: 10m 32s	remaining: 33.2s
9550:	learn: 1495.7698505	test: 2159.2377929	best: 2159.2370279 (9546)	total: 10m 35s	remaining: 29.9s
9600:	learn: 1494.2592864	test: 2159.0066101	best: 2158.9518361 (9593)	total: 10m 39s	remaining: 26.6s
9650:	learn: 1492.8524559	test: 2158.6343614	best: 2158.5382775 (9636)	total: 10m 42s	remaining: 23.2s
9700:	learn: 1491.5750630	test: 2158.2299827	best: 2158.2299827 (9700)	total: 10m 45s	remaining: 19.9s
9750:	learn: 1490.3532089	test: 2157.7408421	best: 2157.7408421 (9750)	total: 10m 49s	remaining: 16.6s
9800:	learn: 1488.7927669	test: 2157.3535994	best: 2157.3535994 (9800)	total: 10m 52s	remaining: 13.2s
9850:	learn: 1487.7713422	test: 2157.1617376	best: 2156.8982718 (9843)	total: 10m 55s	remaining: 9.92s
9900:	learn: 1486.5113913	test: 2156.7532279	best: 2156.7391520 (9893)	total: 10m 59s	remaining: 6.59s
9950:	learn: 1485.0875779	test: 2156.5208211	best: 2156.5147011 (9949)	to

In [12]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.75')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training5_20220204/catboost_quantile075_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 11539.0106592	test: 11825.3894650	best: 11825.3894650 (0)	total: 255ms	remaining: 42m 32s
50:	learn: 4958.9914840	test: 5288.3527021	best: 5288.3527021 (50)	total: 3.55s	remaining: 11m 33s
100:	learn: 3758.1432256	test: 4135.8468857	best: 4135.8468857 (100)	total: 6.87s	remaining: 11m 13s
150:	learn: 3403.1287701	test: 3793.7669148	best: 3793.7669148 (150)	total: 10.3s	remaining: 11m 14s
200:	learn: 3201.6472702	test: 3618.8511458	best: 3618.8511458 (200)	total: 13.8s	remaining: 11m 12s
250:	learn: 3066.5712682	test: 3492.9983409	best: 3492.9983409 (250)	total: 17.1s	remaining: 11m 4s
300:	learn: 2966.5982703	test: 3406.4520275	best: 3406.4520275 (300)	total: 20.4s	remaining: 10m 56s
350:	learn: 2880.4493981	test: 3343.5603559	best: 3343.5603559 (350)	total: 23.6s	remaining: 10m 49s
400:	learn: 2809.3345725	test: 3284.2577468	best: 3284.2577468 (400)	total: 26.9s	remaining: 10m 44s
450:	learn: 2749.8740323	test: 3236.9619270	best

4000:	learn: 1829.8149219	test: 2708.4698041	best: 2708.4698041 (4000)	total: 4m 26s	remaining: 6m 38s
4050:	learn: 1826.2149943	test: 2707.3308758	best: 2707.1561503 (4042)	total: 4m 29s	remaining: 6m 35s
4100:	learn: 1822.2230818	test: 2706.2357053	best: 2706.1543096 (4099)	total: 4m 32s	remaining: 6m 32s
4150:	learn: 1818.5726644	test: 2705.7705868	best: 2705.7360089 (4145)	total: 4m 36s	remaining: 6m 29s
4200:	learn: 1814.8463591	test: 2704.5058929	best: 2704.1818399 (4184)	total: 4m 39s	remaining: 6m 25s
4250:	learn: 1811.1960947	test: 2703.1163513	best: 2703.1019694 (4247)	total: 4m 42s	remaining: 6m 22s
4300:	learn: 1807.5634252	test: 2703.2516029	best: 2702.4131656 (4271)	total: 4m 46s	remaining: 6m 19s
4350:	learn: 1804.1816751	test: 2701.7870916	best: 2701.7870916 (4350)	total: 4m 49s	remaining: 6m 15s
4400:	learn: 1801.2342908	test: 2701.1585696	best: 2701.1585696 (4400)	total: 4m 52s	remaining: 6m 12s
4450:	learn: 1797.2925621	test: 2700.3638902	best: 2700.3470603 (4449)	to

1650:	learn: 2150.6425350	test: 2661.8141343	best: 2661.8074023 (1649)	total: 1m 49s	remaining: 9m 12s
1700:	learn: 2138.5087488	test: 2660.0332200	best: 2660.0124121 (1674)	total: 1m 52s	remaining: 9m 8s
1750:	learn: 2128.9312799	test: 2657.5647260	best: 2657.5647260 (1750)	total: 1m 55s	remaining: 9m 5s
1800:	learn: 2119.0164469	test: 2653.4872891	best: 2653.4872891 (1800)	total: 1m 59s	remaining: 9m 2s
1850:	learn: 2109.2630284	test: 2652.8330632	best: 2652.7455475 (1848)	total: 2m 2s	remaining: 8m 58s
1900:	learn: 2099.6541946	test: 2651.6828129	best: 2651.6828129 (1900)	total: 2m 5s	remaining: 8m 55s
1950:	learn: 2089.5383201	test: 2650.4523909	best: 2650.4523909 (1950)	total: 2m 8s	remaining: 8m 52s
2000:	learn: 2080.9801346	test: 2650.5392946	best: 2649.2853106 (1977)	total: 2m 12s	remaining: 8m 48s
2050:	learn: 2071.7778207	test: 2649.5732557	best: 2649.2853106 (1977)	total: 2m 15s	remaining: 8m 45s
2100:	learn: 2062.3714053	test: 2645.4508370	best: 2645.2430636 (2099)	total: 2

1350:	learn: 2195.6749069	test: 3149.2828052	best: 3149.2828052 (1350)	total: 1m 29s	remaining: 9m 34s
1400:	learn: 2184.1618401	test: 3143.3926302	best: 3143.3926302 (1400)	total: 1m 32s	remaining: 9m 30s
1450:	learn: 2172.6688162	test: 3138.3717984	best: 3138.3717984 (1450)	total: 1m 36s	remaining: 9m 26s
1500:	learn: 2161.5668593	test: 3133.4975401	best: 3133.4975401 (1500)	total: 1m 39s	remaining: 9m 23s
1550:	learn: 2150.7531537	test: 3127.3591998	best: 3127.3591998 (1550)	total: 1m 42s	remaining: 9m 20s
1600:	learn: 2138.2665504	test: 3120.0271574	best: 3119.8824199 (1599)	total: 1m 46s	remaining: 9m 16s
1650:	learn: 2127.7664939	test: 3110.7140774	best: 3110.7140774 (1650)	total: 1m 49s	remaining: 9m 13s
1700:	learn: 2116.3337519	test: 3107.4285660	best: 3107.0656511 (1696)	total: 1m 52s	remaining: 9m 10s
1750:	learn: 2106.2974372	test: 3102.8330728	best: 3102.8330728 (1750)	total: 1m 56s	remaining: 9m 8s
1800:	learn: 2097.3538456	test: 3099.3878298	best: 3099.3878298 (1800)	tot

1950:	learn: 2084.2333650	test: 2575.3611506	best: 2575.3611506 (1950)	total: 2m 9s	remaining: 8m 52s
2000:	learn: 2075.3667323	test: 2570.8571511	best: 2570.8571511 (2000)	total: 2m 12s	remaining: 8m 49s
2050:	learn: 2066.1333595	test: 2566.2284570	best: 2566.2284570 (2050)	total: 2m 15s	remaining: 8m 46s
2100:	learn: 2057.1626870	test: 2561.5423880	best: 2561.1178144 (2094)	total: 2m 19s	remaining: 8m 43s
2150:	learn: 2048.7726480	test: 2555.9744300	best: 2555.9744300 (2150)	total: 2m 22s	remaining: 8m 39s
2200:	learn: 2039.8001395	test: 2553.0201625	best: 2552.5118981 (2197)	total: 2m 25s	remaining: 8m 36s
2250:	learn: 2032.5923284	test: 2547.5877620	best: 2547.3449517 (2249)	total: 2m 29s	remaining: 8m 33s
2300:	learn: 2024.7552299	test: 2544.6812303	best: 2544.4863089 (2296)	total: 2m 32s	remaining: 8m 29s
2350:	learn: 2017.9935656	test: 2540.5598826	best: 2540.5598826 (2350)	total: 2m 35s	remaining: 8m 26s
2400:	learn: 2010.9352352	test: 2535.7325951	best: 2535.7325951 (2400)	tot

5950:	learn: 1712.6972479	test: 2430.9404784	best: 2430.7942109 (5936)	total: 6m 35s	remaining: 4m 29s
6000:	learn: 1710.0876867	test: 2430.3994722	best: 2430.3153224 (5986)	total: 6m 39s	remaining: 4m 25s
6050:	learn: 1707.5102555	test: 2430.1601809	best: 2430.0201864 (6021)	total: 6m 42s	remaining: 4m 22s
6100:	learn: 1705.1249917	test: 2429.3665725	best: 2429.3189896 (6097)	total: 6m 45s	remaining: 4m 19s
6150:	learn: 1702.3277000	test: 2428.5027373	best: 2428.5027373 (6150)	total: 6m 48s	remaining: 4m 15s
6200:	learn: 1699.8489535	test: 2428.6257489	best: 2428.2235132 (6160)	total: 6m 52s	remaining: 4m 12s
6250:	learn: 1697.2904942	test: 2428.5864280	best: 2428.2235132 (6160)	total: 6m 55s	remaining: 4m 9s
6300:	learn: 1695.1208069	test: 2428.0065599	best: 2427.6653710 (6293)	total: 6m 58s	remaining: 4m 5s
6350:	learn: 1692.6596554	test: 2427.5412884	best: 2427.4739686 (6327)	total: 7m 2s	remaining: 4m 2s
6400:	learn: 1690.0395372	test: 2427.5760192	best: 2427.0966718 (6391)	total:

1300:	learn: 2234.2153946	test: 2800.5595588	best: 2800.5595588 (1300)	total: 1m 25s	remaining: 9m 33s
1350:	learn: 2221.9235672	test: 2793.3101188	best: 2793.3101188 (1350)	total: 1m 29s	remaining: 9m 30s
1400:	learn: 2209.0224957	test: 2786.5876104	best: 2786.5876104 (1400)	total: 1m 32s	remaining: 9m 27s
1450:	learn: 2196.6976204	test: 2782.6570436	best: 2782.6570436 (1450)	total: 1m 35s	remaining: 9m 24s
1500:	learn: 2185.2414763	test: 2779.2133225	best: 2779.2133225 (1500)	total: 1m 39s	remaining: 9m 21s
1550:	learn: 2174.8673446	test: 2773.9421708	best: 2773.9421708 (1550)	total: 1m 42s	remaining: 9m 17s
1600:	learn: 2164.5358999	test: 2770.7408015	best: 2770.7408015 (1600)	total: 1m 45s	remaining: 9m 14s
1650:	learn: 2151.9386854	test: 2766.2695332	best: 2766.2695332 (1650)	total: 1m 48s	remaining: 9m 10s
1700:	learn: 2140.4765743	test: 2762.1532246	best: 2762.1532246 (1700)	total: 1m 52s	remaining: 9m 7s
1750:	learn: 2128.7433481	test: 2758.7829434	best: 2758.7829434 (1750)	tot

5300:	learn: 1749.4144231	test: 2650.3843692	best: 2650.3488732 (5288)	total: 5m 50s	remaining: 5m 10s
5350:	learn: 1746.6699179	test: 2650.0373654	best: 2649.8809996 (5338)	total: 5m 53s	remaining: 5m 7s
5400:	learn: 1743.2698099	test: 2648.1208890	best: 2648.1181350 (5399)	total: 5m 57s	remaining: 5m 4s
5450:	learn: 1740.7528150	test: 2648.1392490	best: 2647.6003835 (5416)	total: 6m	remaining: 5m
5500:	learn: 1738.1997122	test: 2646.9415660	best: 2646.9415660 (5500)	total: 6m 3s	remaining: 4m 57s
5550:	learn: 1735.3344904	test: 2645.7056330	best: 2645.6260731 (5542)	total: 6m 7s	remaining: 4m 54s
5600:	learn: 1732.6496550	test: 2644.9920416	best: 2644.9920416 (5600)	total: 6m 10s	remaining: 4m 50s
5650:	learn: 1729.8335461	test: 2643.6340147	best: 2643.6340147 (5650)	total: 6m 13s	remaining: 4m 47s
5700:	learn: 1726.7484640	test: 2642.9146093	best: 2642.6095276 (5687)	total: 6m 16s	remaining: 4m 44s
5750:	learn: 1723.8367304	test: 2642.5896376	best: 2642.1346160 (5734)	total: 6m 20s	

In [13]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.125')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training5_20220204/catboost_quantile0125_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 3771.9033975	test: 3838.4335910	best: 3838.4335910 (0)	total: 262ms	remaining: 43m 38s
50:	learn: 2612.4139906	test: 2675.8053273	best: 2675.8053273 (50)	total: 3.38s	remaining: 10m 58s
100:	learn: 2151.2173545	test: 2210.4397301	best: 2210.4397301 (100)	total: 6.6s	remaining: 10m 47s
150:	learn: 1932.4999907	test: 2001.5856104	best: 2001.5856104 (150)	total: 10s	remaining: 10m 54s
200:	learn: 1809.3666785	test: 1895.5866336	best: 1895.5866336 (200)	total: 13.6s	remaining: 11m
250:	learn: 1730.6863287	test: 1825.0835425	best: 1825.0835425 (250)	total: 17.2s	remaining: 11m 6s
300:	learn: 1675.6112161	test: 1779.0115658	best: 1779.0115658 (300)	total: 20.7s	remaining: 11m 8s
350:	learn: 1632.4107083	test: 1744.0166004	best: 1744.0166004 (350)	total: 24.2s	remaining: 11m 5s
400:	learn: 1594.6513253	test: 1717.3539883	best: 1717.3539883 (400)	total: 27.6s	remaining: 11m
450:	learn: 1566.6147608	test: 1696.6242380	best: 1696.6242380 (

4000:	learn: 1117.5988052	test: 1420.3920421	best: 1420.3920421 (4000)	total: 4m 28s	remaining: 6m 41s
4050:	learn: 1115.5473865	test: 1419.1651391	best: 1419.1651391 (4050)	total: 4m 31s	remaining: 6m 38s
4100:	learn: 1113.4087582	test: 1418.3106368	best: 1418.3106368 (4100)	total: 4m 34s	remaining: 6m 35s
4150:	learn: 1111.1830730	test: 1417.1914454	best: 1417.1914454 (4150)	total: 4m 38s	remaining: 6m 31s
4200:	learn: 1108.8694130	test: 1416.5326295	best: 1416.5326295 (4200)	total: 4m 41s	remaining: 6m 28s
4250:	learn: 1106.7980280	test: 1415.4322571	best: 1415.4322571 (4250)	total: 4m 44s	remaining: 6m 25s
4300:	learn: 1104.6580991	test: 1414.8268380	best: 1414.8268380 (4300)	total: 4m 48s	remaining: 6m 22s
4350:	learn: 1102.7764335	test: 1414.1212043	best: 1414.0619935 (4345)	total: 4m 51s	remaining: 6m 18s
4400:	learn: 1100.7817777	test: 1413.3897137	best: 1413.3427428 (4396)	total: 4m 55s	remaining: 6m 15s
4450:	learn: 1098.8251424	test: 1412.8679853	best: 1412.8679853 (4450)	to

1150:	learn: 1386.3656272	test: 1561.0328614	best: 1561.0328614 (1150)	total: 1m 16s	remaining: 9m 50s
1200:	learn: 1377.0871195	test: 1553.0381782	best: 1553.0381782 (1200)	total: 1m 20s	remaining: 9m 46s
1250:	learn: 1368.3262095	test: 1547.5401409	best: 1547.5401409 (1250)	total: 1m 23s	remaining: 9m 43s
1300:	learn: 1359.4802317	test: 1540.4538157	best: 1540.4538157 (1300)	total: 1m 26s	remaining: 9m 40s
1350:	learn: 1350.5958510	test: 1533.6658395	best: 1533.6658395 (1350)	total: 1m 30s	remaining: 9m 36s
1400:	learn: 1342.5027842	test: 1528.0836525	best: 1528.0836525 (1400)	total: 1m 33s	remaining: 9m 32s
1450:	learn: 1333.5832136	test: 1520.7695110	best: 1520.7695110 (1450)	total: 1m 36s	remaining: 9m 28s
1500:	learn: 1326.3896314	test: 1515.4628761	best: 1515.4628761 (1500)	total: 1m 39s	remaining: 9m 25s
1550:	learn: 1320.1711157	test: 1511.0941282	best: 1511.0941282 (1550)	total: 1m 43s	remaining: 9m 21s
1600:	learn: 1314.4467887	test: 1507.3771581	best: 1507.3771581 (1600)	to

5150:	learn: 1080.7613212	test: 1390.9671768	best: 1390.9671768 (5150)	total: 5m 41s	remaining: 5m 21s
5200:	learn: 1078.9358830	test: 1390.8088223	best: 1390.5153693 (5187)	total: 5m 45s	remaining: 5m 18s
5250:	learn: 1077.1949006	test: 1390.0736597	best: 1390.0436718 (5243)	total: 5m 48s	remaining: 5m 15s
5300:	learn: 1075.7632833	test: 1389.3030012	best: 1389.3030012 (5300)	total: 5m 51s	remaining: 5m 11s
5350:	learn: 1074.1482961	test: 1388.4474279	best: 1388.4388600 (5348)	total: 5m 55s	remaining: 5m 8s
5400:	learn: 1072.5642147	test: 1387.8022290	best: 1387.7893771 (5392)	total: 5m 58s	remaining: 5m 5s
5450:	learn: 1071.2452052	test: 1387.0456464	best: 1387.0456464 (5450)	total: 6m 2s	remaining: 5m 2s
5500:	learn: 1069.5979350	test: 1385.9047291	best: 1385.8759652 (5499)	total: 6m 5s	remaining: 4m 58s
5550:	learn: 1068.2570465	test: 1385.7324517	best: 1385.6912948 (5546)	total: 6m 8s	remaining: 4m 55s
5600:	learn: 1066.9065190	test: 1385.2240343	best: 1385.2240343 (5600)	total: 6

9200:	learn: 987.7349641	test: 1356.2604625	best: 1356.2604625 (9200)	total: 10m 14s	remaining: 53.4s
9250:	learn: 986.9700426	test: 1355.8969357	best: 1355.8952527 (9248)	total: 10m 17s	remaining: 50s
9300:	learn: 986.0743064	test: 1355.9904184	best: 1355.8790348 (9251)	total: 10m 21s	remaining: 46.7s
9350:	learn: 985.4274241	test: 1355.7578592	best: 1355.7578592 (9350)	total: 10m 24s	remaining: 43.4s
9400:	learn: 984.7351774	test: 1355.4959240	best: 1355.4766461 (9384)	total: 10m 28s	remaining: 40s
9450:	learn: 984.1030596	test: 1355.3869883	best: 1355.3672514 (9444)	total: 10m 31s	remaining: 36.7s
9500:	learn: 983.4390413	test: 1355.3791854	best: 1355.3672514 (9444)	total: 10m 34s	remaining: 33.3s
9550:	learn: 982.6429844	test: 1355.1060813	best: 1355.0903223 (9540)	total: 10m 38s	remaining: 30s
9600:	learn: 981.6733494	test: 1355.1213812	best: 1355.0903223 (9540)	total: 10m 41s	remaining: 26.7s
9650:	learn: 981.0331226	test: 1354.8323651	best: 1354.8014592 (9647)	total: 10m 44s	rem

...

7150:	learn: 1015.7628841	test: 1408.9590009	best: 1408.9019321 (7142)	total: 7m 56s	remaining: 3m 9s
7200:	learn: 1014.8640879	test: 1408.5841521	best: 1408.5841521 (7200)	total: 7m 59s	remaining: 3m 6s
7250:	learn: 1013.9096764	test: 1407.9622091	best: 1407.9622091 (7250)	total: 8m 2s	remaining: 3m 3s
7300:	learn: 1012.7434370	test: 1407.9126373	best: 1407.9126373 (7300)	total: 8m 6s	remaining: 2m 59s
7350:	learn: 1011.6940128	test: 1407.4860747	best: 1407.4747527 (7344)	total: 8m 9s	remaining: 2m 56s
7400:	learn: 1010.8203085	test: 1407.2152655	best: 1407.0968439 (7381)	total: 8m 12s	remaining: 2m 53s
7450:	learn: 1009.8339201	test: 1406.8391927	best: 1406.8391927 (7450)	total: 8m 16s	remaining: 2m 49s
7500:	learn: 1008.8868526	test: 1406.3671893	best: 1406.3232784 (7497)	total: 8m 19s	remaining: 2m 46s
7550:	learn: 1007.7734745	test: 1406.3000225	best: 1406.3000225 (7550)	total: 8m 22s	remaining: 2m 43s
7600:	learn: 1006.8853884	test: 1405.7902281	best: 1405.7818131 (7598)	total: 8

1100:	learn: 1376.3984683	test: 1502.9322164	best: 1502.9322164 (1100)	total: 1m 13s	remaining: 9m 52s
1150:	learn: 1368.0223523	test: 1499.8711553	best: 1499.8711553 (1150)	total: 1m 16s	remaining: 9m 48s
1200:	learn: 1358.9559020	test: 1494.5867053	best: 1494.5867053 (1200)	total: 1m 19s	remaining: 9m 45s
1250:	learn: 1349.9529466	test: 1490.3126641	best: 1490.3109811 (1247)	total: 1m 23s	remaining: 9m 41s
1300:	learn: 1340.8969004	test: 1486.6506208	best: 1486.6506208 (1300)	total: 1m 26s	remaining: 9m 38s
1350:	learn: 1333.1918406	test: 1482.5480920	best: 1482.5480920 (1350)	total: 1m 29s	remaining: 9m 34s
1400:	learn: 1325.6122404	test: 1478.5860168	best: 1478.5616899 (1399)	total: 1m 32s	remaining: 9m 30s
1450:	learn: 1318.9861338	test: 1475.5713145	best: 1475.5713145 (1450)	total: 1m 36s	remaining: 9m 27s
1500:	learn: 1311.5823302	test: 1472.8587862	best: 1472.8587862 (1500)	total: 1m 39s	remaining: 9m 23s
1550:	learn: 1304.8394849	test: 1470.0188094	best: 1469.9712265 (1545)	to

200:	learn: 1825.3121238	test: 1822.9033734	best: 1822.9033734 (200)	total: 13.5s	remaining: 10m 57s
250:	learn: 1745.7126000	test: 1761.2664565	best: 1761.2664565 (250)	total: 17s	remaining: 10m 59s
300:	learn: 1694.5630286	test: 1723.7591495	best: 1723.7591495 (300)	total: 20.3s	remaining: 10m 55s
350:	learn: 1654.6368891	test: 1694.6809116	best: 1694.6809116 (350)	total: 23.7s	remaining: 10m 51s
400:	learn: 1621.1566189	test: 1669.2172957	best: 1669.2172957 (400)	total: 27.1s	remaining: 10m 48s
450:	learn: 1592.8849109	test: 1649.2662652	best: 1649.2662652 (450)	total: 30.4s	remaining: 10m 44s
500:	learn: 1569.2789229	test: 1632.4046082	best: 1632.4046082 (500)	total: 33.8s	remaining: 10m 40s
550:	learn: 1545.5389073	test: 1617.0492349	best: 1617.0492349 (550)	total: 37.1s	remaining: 10m 36s
600:	learn: 1527.7531735	test: 1604.8263810	best: 1604.8263810 (600)	total: 40.5s	remaining: 10m 32s
650:	learn: 1509.9333209	test: 1593.8142151	best: 1593.8142151 (650)	total: 43.8s	remaining: 

4250:	learn: 1112.6219812	test: 1421.4733218	best: 1421.3402120 (4245)	total: 4m 42s	remaining: 6m 21s
4300:	learn: 1110.3046507	test: 1420.3581058	best: 1420.3581058 (4300)	total: 4m 45s	remaining: 6m 18s
4350:	learn: 1108.1188232	test: 1420.1073390	best: 1420.0837770 (4328)	total: 4m 49s	remaining: 6m 15s
4400:	learn: 1105.7613303	test: 1418.9457639	best: 1418.9267920 (4395)	total: 4m 52s	remaining: 6m 11s
4450:	learn: 1103.8238976	test: 1418.4353564	best: 1418.4353564 (4450)	total: 4m 55s	remaining: 6m 8s
4500:	learn: 1101.6750195	test: 1417.6044140	best: 1417.5901851 (4499)	total: 4m 59s	remaining: 6m 5s
4550:	learn: 1099.4744276	test: 1416.9145376	best: 1416.9145376 (4550)	total: 5m 2s	remaining: 6m 2s
4600:	learn: 1097.3703783	test: 1416.9099476	best: 1416.8192187 (4581)	total: 5m 5s	remaining: 5m 58s
4650:	learn: 1095.4812934	test: 1416.1862582	best: 1416.1862582 (4650)	total: 5m 9s	remaining: 5m 55s
4700:	learn: 1093.6647303	test: 1415.7414876	best: 1415.6674357 (4694)	total: 5

8250:	learn: 1000.4199454	test: 1393.0447214	best: 1393.0115204 (8248)	total: 9m 9s	remaining: 1m 56s
8300:	learn: 999.5554217	test: 1392.6166277	best: 1392.5751648 (8296)	total: 9m 13s	remaining: 1m 53s
8350:	learn: 998.8293625	test: 1392.4732669	best: 1392.4715839 (8349)	total: 9m 16s	remaining: 1m 49s
8400:	learn: 997.9382169	test: 1391.9377673	best: 1391.9377673 (8400)	total: 9m 19s	remaining: 1m 46s
8450:	learn: 997.2829200	test: 1391.9284343	best: 1391.8058814 (8425)	total: 9m 23s	remaining: 1m 43s
8500:	learn: 996.2983682	test: 1391.7261685	best: 1391.6626735 (8481)	total: 9m 26s	remaining: 1m 39s
8550:	learn: 995.4375929	test: 1391.5716386	best: 1391.5688846 (8549)	total: 9m 29s	remaining: 1m 36s
8600:	learn: 994.7232382	test: 1391.5823486	best: 1391.5064607 (8559)	total: 9m 33s	remaining: 1m 33s
8650:	learn: 993.9994741	test: 1391.3312758	best: 1391.2890478 (8645)	total: 9m 36s	remaining: 1m 29s
8700:	learn: 993.1351798	test: 1390.9299571	best: 1390.9299571 (8700)	total: 9m 39

In [14]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.875')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training5_20220204/catboost_quantile0875_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 9303.2721953	test: 9465.0991102	best: 9465.0991102 (0)	total: 168ms	remaining: 28m 2s
50:	learn: 3585.7650034	test: 3919.2644549	best: 3919.2644549 (50)	total: 3.49s	remaining: 11m 21s
100:	learn: 2602.6082463	test: 2941.8454609	best: 2941.8454609 (100)	total: 6.68s	remaining: 10m 54s
150:	learn: 2323.5764517	test: 2676.3536775	best: 2676.3536775 (150)	total: 10.2s	remaining: 11m 3s
200:	learn: 2193.6707652	test: 2559.6338722	best: 2559.6338722 (200)	total: 13.6s	remaining: 11m 4s
250:	learn: 2097.9907830	test: 2478.5892680	best: 2478.5892680 (250)	total: 17s	remaining: 11m 1s
300:	learn: 2025.3341804	test: 2419.3493887	best: 2419.3493887 (300)	total: 20.3s	remaining: 10m 55s
350:	learn: 1971.5993443	test: 2372.9977289	best: 2372.9977289 (350)	total: 23.5s	remaining: 10m 47s
400:	learn: 1925.7187425	test: 2336.6337240	best: 2336.6337240 (400)	total: 26.9s	remaining: 10m 43s
450:	learn: 1883.9593380	test: 2308.1611276	best: 2308.1

4000:	learn: 1234.8767463	test: 1977.7791930	best: 1977.2349738 (3950)	total: 4m 27s	remaining: 6m 40s
4050:	learn: 1232.0392157	test: 1976.3846025	best: 1976.2716889 (4047)	total: 4m 30s	remaining: 6m 37s
4100:	learn: 1229.6008611	test: 1975.9015831	best: 1975.8474212 (4098)	total: 4m 33s	remaining: 6m 33s
4150:	learn: 1226.7374736	test: 1976.3875095	best: 1975.8474212 (4098)	total: 4m 36s	remaining: 6m 30s
4200:	learn: 1224.0371837	test: 1975.7840794	best: 1975.5149533 (4177)	total: 4m 40s	remaining: 6m 26s
4250:	learn: 1221.4413925	test: 1975.3114640	best: 1975.3114640 (4250)	total: 4m 43s	remaining: 6m 23s
4300:	learn: 1218.3608986	test: 1975.2084953	best: 1974.3778610 (4286)	total: 4m 46s	remaining: 6m 20s
4350:	learn: 1215.8396947	test: 1974.6013933	best: 1974.3778610 (4286)	total: 4m 50s	remaining: 6m 16s
4400:	learn: 1213.0281740	test: 1973.5919025	best: 1973.5919025 (4400)	total: 4m 53s	remaining: 6m 13s
4450:	learn: 1210.2598758	test: 1972.8865748	best: 1972.7392362 (4442)	to

150:	learn: 2339.3508633	test: 2779.3988458	best: 2779.3988458 (150)	total: 10.1s	remaining: 10m 58s
200:	learn: 2204.2014216	test: 2691.2821454	best: 2691.2821454 (200)	total: 13.6s	remaining: 11m
250:	learn: 2106.8864857	test: 2619.3035271	best: 2619.3035271 (250)	total: 16.8s	remaining: 10m 52s
300:	learn: 2031.1317942	test: 2566.1882181	best: 2566.1596072 (299)	total: 20.1s	remaining: 10m 48s
350:	learn: 1972.3040604	test: 2536.6456962	best: 2536.6456962 (350)	total: 23.4s	remaining: 10m 42s
400:	learn: 1920.4881452	test: 2509.6034922	best: 2509.3519610 (399)	total: 26.7s	remaining: 10m 39s
450:	learn: 1866.8787687	test: 2486.6018523	best: 2486.3871939 (449)	total: 30.1s	remaining: 10m 36s
500:	learn: 1828.0929114	test: 2470.4512242	best: 2470.4512242 (500)	total: 33.5s	remaining: 10m 35s
550:	learn: 1794.4836425	test: 2463.3544951	best: 2463.3544951 (550)	total: 36.9s	remaining: 10m 32s
600:	learn: 1760.3463720	test: 2452.6491195	best: 2452.6491195 (600)	total: 40.2s	remaining: 10

3050:	learn: 1311.5401023	test: 1769.8943252	best: 1769.5496173 (3036)	total: 3m 22s	remaining: 7m 40s
3100:	learn: 1307.9614417	test: 1768.7425449	best: 1768.7425449 (3100)	total: 3m 25s	remaining: 7m 36s
3150:	learn: 1304.3065873	test: 1768.3510191	best: 1767.4862659 (3148)	total: 3m 28s	remaining: 7m 33s
3200:	learn: 1300.8916362	test: 1767.0678123	best: 1767.0678123 (3200)	total: 3m 32s	remaining: 7m 30s
3250:	learn: 1297.3264396	test: 1766.5841808	best: 1765.7746604 (3216)	total: 3m 35s	remaining: 7m 26s
3300:	learn: 1293.8788996	test: 1765.5642861	best: 1765.5217522 (3297)	total: 3m 38s	remaining: 7m 23s
3350:	learn: 1290.4007597	test: 1765.8430512	best: 1765.5217522 (3297)	total: 3m 41s	remaining: 7m 20s
3400:	learn: 1286.7677843	test: 1765.0509727	best: 1764.8404454 (3368)	total: 3m 45s	remaining: 7m 17s
3450:	learn: 1282.7717407	test: 1763.6834631	best: 1763.6643382 (3449)	total: 3m 48s	remaining: 7m 13s
3500:	learn: 1279.3538827	test: 1763.1285339	best: 1763.1031360 (3499)	to

2050:	learn: 1404.0697487	test: 2041.1202077	best: 2040.6898191 (2040)	total: 2m 15s	remaining: 8m 45s
2100:	learn: 1398.8302805	test: 2041.0173918	best: 2040.6235701 (2063)	total: 2m 18s	remaining: 8m 42s
2150:	learn: 1392.7916960	test: 2039.7601918	best: 2039.3265902 (2125)	total: 2m 22s	remaining: 8m 38s
2200:	learn: 1386.2624420	test: 2039.3232242	best: 2039.0305354 (2179)	total: 2m 25s	remaining: 8m 35s
2250:	learn: 1380.8311124	test: 2043.3465089	best: 2038.6731277 (2213)	total: 2m 28s	remaining: 8m 32s
2300:	learn: 1375.1109199	test: 2043.6272636	best: 2038.6731277 (2213)	total: 2m 32s	remaining: 8m 29s
2350:	learn: 1369.8526328	test: 2041.8749561	best: 2038.6731277 (2213)	total: 2m 35s	remaining: 8m 25s
2400:	learn: 1364.3478635	test: 2039.5516530	best: 2038.6731277 (2213)	total: 2m 38s	remaining: 8m 22s
bestTest = 2038.673128
bestIteration = 2213
Shrink model to first 2214 iterations.
-------------- Fold 4 ending ---------------
